In [8]:
pip install scikit-learn==1.5.2

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install xgboost==2.1.1

Note: you may need to restart the kernel to use updated packages.


In [9]:
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [10]:
hdb_df = pd.read_csv('/kaggle/input/hdb-dataset/hdb_combined_clean.csv', low_memory=False)

In [34]:
sample_df = hdb_df.sample(frac=0.1, random_state=42)
sample_df.shape

(90679, 62)

In [35]:
# Split features and target (assuming 'resale_price' is the target)
X = sample_df.drop(columns=['resale_price'])
y = sample_df['resale_price']

# Train-test split (80-20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaling features (only for models sensitive to scale like linear regression or neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# To calculate adjusted r2 later
n = len(y_test)  
p = X_train_scaled.shape[1]


In [47]:
print('hello')

hello


### Decision Tree

In [18]:
# Define the parameter grid to tune the hyperparameters
param_grid = {
    'max_depth': [10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
dtree_reg = DecisionTreeRegressor(random_state=42) # Initialize a decision tree regressor
grid_search = GridSearchCV(estimator=dtree_reg, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train_scaled, y_train)
best_dtree_reg = grid_search.best_estimator_ # Get the best estimator from the grid search
y_pred = best_dtree_reg.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")
print(f"Test RMSE: {rmse}")
print(f"Test R²: {r2_score(y_test, y_pred):.4f}")


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=2; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=10; total time=   0.7s
[CV] END max_depth=10, min_samples_leaf=1, min_samples_split=5; total time=   0.7s
[CV] END max_depth=10, m

In [31]:
# Decision Tree Regressor
dt = DecisionTreeRegressor(max_depth= 20, min_samples_leaf=2, min_samples_split=10)
dt.fit(X_train_scaled, y_train)
dt_pred = dt.predict(X_test_scaled)

r_squared = r2_score(y_test, dt_pred)
adjusted_r_squared = 1 - ((1 - r_squared) * (n - 1) / (n - p - 1))
rmse = np.sqrt(mean_squared_error(y_test, dt_pred))
mae = mean_absolute_error(y_test, dt_pred)

print("Decision Tree:")
print(f"  R²: {r_squared:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

Decision Tree:
  R²: 0.9507
  RMSE: 37371.2321
  MAE: 24869.3571


### Random Forest

In [23]:
# Random Forest Regressor
rf = RandomForestRegressor(n_estimators=200, max_depth=10, random_state=42)
rf.fit(X_train_scaled, y_train)
rf_pred = rf.predict(X_test_scaled)

r_squared = r2_score(y_test, rf_pred)
rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
mae = mean_absolute_error(y_test, rf_pred)

print("Random Forest:")
print(f"  R²: {r_squared:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

Random Forest:
  R²: 0.9445
  RMSE: 39655.1124
  MAE: 28129.3877


In [19]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from tqdm import tqdm

# Define a reduced parameter grid
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
}

# Initialize Random Forest model
rf_model = RandomForestRegressor(random_state=42)

# Set up randomized search
random_search = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, 
                                   n_iter=10, scoring='neg_mean_squared_error', 
                                   cv=3, n_jobs=-1, verbose=2, random_state=42)

# Fit randomized search
random_search.fit(X_train_scaled, y_train)

# Evaluate the best model
best_rf_model = random_search.best_estimator_
y_pred = best_rf_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation Score:", random_search.best_score_)

/opt/conda/lib/python3.10/site-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=10. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=  38.9s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=  39.1s
[CV] END max_depth=10, min_samples_split=2, n_estimators=100; total time=  39.5s
[CV] END max_depth=10, min_samples_split=2, n_estimators=200; total time= 1.3min
[CV] END max_depth=10, min_samples_split=5, n_estimators=100; total time=  37.9s
[CV] END max_depth=10, min_samples_split=2, n_estimators=200; total time= 1.3min
[CV] END max_depth=10, min_samples_split=5, n_estimators=100; total time=  38.5s
[CV] END max_depth=10, min_samples_split=5, n_estimators=100; total time=  38.5s
[CV] END max_depth=10, min_samples_split=2, n_estimators=200; total time= 1.3min
[CV] END max_depth=20, min_samples_split=2, n_estimators=100; total time= 1.1min
[CV] END max_depth=10, min_samples_split=5, n_estimators=200; total time= 1.3min
[CV] END max_depth=10, min_samples_split=5, n_est

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END max_depth=20, min_samples_split=2, n_estimators=200; total time= 2.3min
[CV] END max_depth=20, min_samples_split=5, n_estimators=100; total time= 1.1min
[CV] END max_depth=20, min_samples_split=2, n_estimators=200; total time= 2.3min
[CV] END max_depth=20, min_samples_split=5, n_estimators=100; total time= 1.1min
[CV] END max_depth=20, min_samples_split=5, n_estimators=200; total time= 2.0min
[CV] END max_depth=20, min_samples_split=5, n_estimators=200; total time= 1.6min
[CV] END max_depth=20, min_samples_split=5, n_estimators=200; total time= 1.7min
Mean Squared Error: 848090436.7709485
Best Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 20}
Best Cross-Validation Score: -919940650.0316023


In [20]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse) 
r_squared = r2_score(y_test, y_pred)

# Print results
print(f"Best Parameters: {random_search.best_params_}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r_squared:.4f}")

Best Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 20}
Mean Squared Error: 848090436.7709
Root Mean Squared Error: 29121.9923
R² Score: 0.9700


### Gradient Boosting

In [24]:
# Gradient Boosting Regressor
gb = GradientBoostingRegressor(random_state=42)
gb.fit(X_train_scaled, y_train)
gb_pred = gb.predict(X_test_scaled)

r_squared = r2_score(y_test, gb_pred)
rmse = np.sqrt(mean_squared_error(y_test, gb_pred))
mae = mean_absolute_error(y_test, gb_pred)

print("Gradient Boosting:")
print(f"  R²: {r_squared:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

Gradient Boosting:
  R²: 0.9424
  RMSE: 40372.9122
  MAE: 28750.9431


In [36]:
gb = GradientBoostingRegressor()

# Define the parameter grid for Grid Search
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
}

grid_search = GridSearchCV(estimator=gb, param_grid=param_grid, 
                           scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

y_pred = grid_search.best_estimator_.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test, y_pred)

# Print results
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r_squared:.4f}")


Fitting 3 folds for each of 9 candidates, totalling 27 fits
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  21.7s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  21.8s
[CV] END ...............learning_rate=0.01, n_estimators=100; total time=  22.2s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  43.5s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  44.8s
[CV] END ...............learning_rate=0.01, n_estimators=200; total time=  45.6s
[CV] END ...............learning_rate=0.01, n_estimators=300; total time= 1.1min
[CV] END ...............learning_rate=0.05, n_estimators=100; total time=  23.9s
[CV] END ...............learning_rate=0.01, n_estimators=300; total time= 1.1min
[CV] END ...............learning_rate=0.05, n_estimators=100; total time=  22.1s
[CV] END ...............learning_rate=0.05, n_estimators=100; total time=  22.1s
[CV] END ...............learning_rate=0.01, n_est

### XGBoost

In [40]:
# XGBoost Regressor
xgb = XGBRegressor(n_estimators=200, max_depth=10, random_state=42)
xgb.fit(X_train_scaled, y_train)
xgb_pred = xgb.predict(X_test_scaled)

r_squared = r2_score(y_test, xgb_pred)
rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
mae = mean_absolute_error(y_test, xgb_pred)

print("XGBoost:")
print(f"  R²: {r_squared:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

XGBoost:
  R²: 0.9735
  RMSE: 27388.1836
  MAE: 19549.3749


In [43]:
xgb = XGBRegressor()

# Define the parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],         
    'learning_rate': [0.01, 0.1],       
    'max_depth': [10, 15]              
}

grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, n_jobs=-1, verbose=2)
grid_search.fit(X_train_scaled, y_train)

best_xgb_model = grid_search.best_estimator_

y_pred = best_xgb_model.predict(X_test_scaled)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)  
r_squared = r2_score(y_test, y_pred)

print(f"Best Parameters: {grid_search.best_params_}")
print(f"Mean Squared Error: {mse:.4f}")
print(f"Root Mean Squared Error: {rmse:.4f}")
print(f"R² Score: {r_squared:.4f}")


Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=100; total time=   9.8s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=100; total time=   9.8s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=100; total time=  10.0s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=200; total time=  20.0s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=200; total time=  19.2s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=200; total time=  19.8s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=300; total time=  29.1s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=300; total time=  29.0s
[CV] END .learning_rate=0.01, max_depth=10, n_estimators=300; total time=  29.3s
[CV] END .learning_rate=0.01, max_depth=15, n_estimators=100; total time=  29.4s
[CV] END .learning_rate=0.01, max_depth=15, n_estimators=100; total time=  29.1s
[CV] END .learning_rate=0.01, max_depth=15, n_es

/opt/conda/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


[CV] END .learning_rate=0.01, max_depth=15, n_estimators=200; total time= 1.1min
[CV] END .learning_rate=0.01, max_depth=15, n_estimators=200; total time= 1.0min
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=100; total time=   8.1s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=100; total time=   8.4s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=100; total time=   7.4s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=200; total time=  12.8s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=200; total time=  12.2s
[CV] END .learning_rate=0.01, max_depth=15, n_estimators=300; total time= 1.8min
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=200; total time=  12.0s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=300; total time=  16.9s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=300; total time=  17.5s
[CV] END ..learning_rate=0.1, max_depth=10, n_estimators=300; total time=  17.0s
[CV] END .learning_rate=0.01

### Stacking

In [ ]:
dt_pred = dt.predict(X_test_scaled)
rf_pred = rf.predict(X_test_scaled)
gb_pred = gb.predict(X_test_scaled)
xgb_pred = xgb.predict(X_test_scaled)

In [ ]:
# Combine the predictions of the base models into a single feature matrix
X_test_meta = np.column_stack((dt_pred, rf_pred, gb_pred, xgb_pred, knn_pred))

meta_model = LinearRegression()
meta_model.fit(X_test_meta, y_test)

In [ ]:
# Make predictions on new data
dt_pred_new = dt.predict(X_test_scaled)
rf_pred_new = rf.predict(X_test_scaled)
gb_pred_new = gb.predict(X_test_scaled)
xgb_pred_new = xgb.predict(X_test_scaled)
knn_pred_new = knn.predict(X_test_scaled)

X_new_meta = np.column_stack((dt_pred_new, rf_pred_new, gb_pred_new, xgb_pred_new, knn_pred_new))

y_new_pred = meta_model.predict(X_new_meta)

r_squared = r2_score(y_test, y_new_pred)
adjusted_r_squared = 1 - ((1 - r_squared) * (n - 1) / (n - p - 1))
rmse = np.sqrt(mean_squared_error(y_test, y_new_pred))
mae = mean_absolute_error(y_test, y_new_pred)

print(f"  R²: {r_squared:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

In [ ]:
import joblib

# Save the meta-model
joblib.dump(meta_model, 'meta_model.pkl')

# Save the base models 
joblib.dump(dt, 'decision_tree_model.pkl')
joblib.dump(rf, 'random_forest_model.pkl')
joblib.dump(gb, 'gradient_boosting_model.pkl')
joblib.dump(xgb, 'xgboost_model.pkl')
joblib.dump(lr, 'linear_regression_model.pkl')
joblib.dump(knn, 'knn_model.pkl')
